In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import json
import requests
import re
import time
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2

In [2]:
class HH_vacancies:
    def __init__(self, ):
        res_df  = self.search_areas_ids()
        self.areas_ids = res_df['id_y'].unique().tolist()
        
    def search_areas_ids(self):
        #строка запроса, где 113 - код России
        url = 'https://api.hh.ru/areas/113'
        # отправляю запрос и полученный ответ преобразовываю в json
        result = requests.get(url).json()['areas']
        # преобразовываю в DataFrame для удобства дальнейшей работы с данными
        df = pd.json_normalize(result)
        # произвожу поиск вложенных areas для parent_ids
        areas_list = []
        for item in df['areas'].tolist():
            areas_list += item
        df2 = pd.json_normalize(areas_list)
        res_df = df.drop('areas', axis=1).merge(
                                                df2.drop('areas', axis=1),
                                                left_on='id',
                                                right_on='parent_id',
                                                how='inner'
                                            )
        # удаляю колонки, дублирующие данные
        res_df = res_df.drop(columns = ['parent_id_x','parent_id_y'], axis=1)
        return res_df

    def search_vacancies_ids(self, start, stop, filename):
        del_columns = [
            'address', 'response_url', 'sort_point_distance', 'relations',
            'contacts', 'working_days', 'working_time_intervals', 'working_time_modes',
            'accept_temporary', 'address.building', 'address.description',
            'address.metro', 'address.metro_stations', 'employer.logo_urls.240',
            'employer.logo_urls.90', 'employer.logo_urls.original', 'address.metro.station_name',
            'address.metro.line_name', 'address.metro.station_id', 'address.metro.line_id',
            'address.metro.lat', 'address.metro.lng', 'salary', 'insider_interview.id',
            'insider_interview.url'
        ]
        res_frame = pd.DataFrame()
        for area_id in tqdm(self.areas_ids[start:stop]):
            try:
                for page in range(1,100):
                    url = f'https://api.hh.ru/vacancies?page={page}&per_page=100&area={area_id}'
                    time.sleep(0.5)
                    req = requests.get(url).json()
                    new_frame = pd.json_normalize(req['items'])
                    res_frame = pd.concat([res_frame, new_frame])
                    if len(req['items'])==0:
                        break
            except Exception as error:
                continue
        res_frame = res_frame.drop(columns = del_columns)
        res_frame.to_csv(filename, sep=';', index = False)
    
    def get_vacancies_data(self, vacancies_ids, left, right):
        res_df = pd.DataFrame()
        count = 0
        for vacancy_id in tqdm(vacancies_ids[left : right]):
            try:
                url = f'https://api.hh.ru/vacancies/{vacancy_id}'
                res = requests.get(url)
                if res.status_code == 200 and len(res.text)>10:
                    df = pd.json_normalize(res.json())
                    res_df = pd.concat([res_df, df])
                    count += 1
                    if count == 100:
                        time.sleep(5)
                        count = 0
                else:
                    time.sleep(2)
                    raise Exception('Not 200 status')
            except Exception as error:
                continue
        res_df.to_excel(f'{left}-{right}.xlsx')

In [3]:
hh = HH_vacancies()

In [ ]:
limits = [
    [600, 650],[650, 700],[700, 750]
]
for limit in limits:
    hh.search_vacancies_ids(limit[0], limit[1], f'areas_{limit[0]}_{limit[1]}.csv')

In [ ]:
ids_df = pd.read_csv('Result_vacancies_ids.csv', sep=';')
ids = ids_df['id'].tolist()
hh.get_vacancies_data(ids, 25000, 26000)

In [ ]:
excel_files = [file for file in os.listdir() if file[-4:]=='xlsx']
columns = [
    'id','name','description','key_skills','specializations','professional_roles',
    'published_at','apply_alternate_url','alternate_url','area.id','area.name',
    'salary.from','salary.to','salary.currency','experience.id','experience.name',
    'schedule.name','employment.name','employer.id','employer.name','address.city',
    'address.street','address.building','address.lat','address.lng','address.raw'
]
res_df = pd.DataFrame()
for excel_file in excel_files:
    try:
        df = pd.read_excel(excel_file)[columns]
        res_df = pd.concat([res_df, df])
    except Exception as error:
        print(f'{error}-{excel_file}')
res_df.to_excel('Result_vac.xlsx')

In [4]:
class Similarity:
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        additional_stopwords = ['quot']
        self.del_words = stopwords.words('russian') + additional_stopwords
    
    def del_tags(self, text):
        """
        Метод для очистки входного текста от html-тегов.
        input:  text - текст для очистки.
        output: очищенная строка.
        """
        return re.sub('\<.{1,9}\>',' ',text)
    
    def clean_text(self, dirty_text):
        """
        Метод для очистки текста.
        input:  dirty_text - текст для очистки.
        output: result_text - токенизированный текст без стоп-слов и в нормальной форме.
        """
        sub_text = re.sub('[^А-Яа-яA-Za-z\s\|]|\s{2,}',' ', self.del_tags(dirty_text) ).strip().lower()
        tokens = word_tokenize(str(sub_text))
        clean_tokens = tuple(map(lambda x: self.morph.parse(x)[0].normal_form if x not in self.del_words else '', tokens ))
        result_text = ' '.join(clean_tokens)
        return result_text
    
    def vectorize_texts(self, texts_for_vectorized, minimal_df):
        """
        Метод для векторизации списка текстов.
        input:  texts_for_vectorized - список текстов для векторизации.
        output: vectors_list - список векторов на основе представленных текстов.
        """
        vectorizer = CountVectorizer(min_df = minimal_df)
        vectorizer.fit(texts_for_vectorized)
        print(f'Количество слов в словаре {vectorizer.vocabulary_}')
        x = vectorizer.transform(texts_for_vectorized)
        vectors = x.toarray()
        vectors_list = [vector for vector in vectors]
        return vectors_list

In [5]:
sim = Similarity()

In [6]:
df = pd.read_excel('Result_vac.xlsx')

In [7]:
df.head()

,Unnamed: 0,id,name,description,key_skills,specializations,professional_roles,published_at,apply_alternate_url,alternate_url,...,schedule.name,employment.name,employer.id,employer.name,address.city,address.street,address.building,address.lat,address.lng,address.raw
0,0,68153788.0,Преподаватель по классу гитары,<strong>Обязанности:</strong> <ul> <li>препода...,"[{'name': 'Грамотная речь'}, {'name': 'Способн...","[{'id': '14.60', 'name': 'Гуманитарные науки',...","[{'id': '132', 'name': 'Учитель, преподаватель...",2022-07-26T13:24:15+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68153788,...,Полный день,Полная занятость,9047939.0,МУДО Егорьевская детская школа искусств,NaN,NaN,NaN,NaN,NaN,NaN
1,1,68357063.0,Продавец,<p><em>Вакансии открыты в разных микрорайонах ...,[],"[{'id': '17.302', 'name': 'FMCG, Товары народн...","[{'id': '97', 'name': 'Продавец-консультант, п...",2022-08-15T10:24:41+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68357063,...,Сменный график,Полная занятость,49357.0,"МАГНИТ, Розничная сеть",Шатура,Маршала Борзова,10,55.579923,39.521530,"Шатура, Маршала Борзова, 10"
2,2,67921742.0,Врач-эндокринолог (г. Солнечногорск),<p><strong><em>Министерство Здравоохранения МО...,"[{'name': 'Пользователь ПК'}, {'name': 'Работа...","[{'id': '13.587', 'name': 'Врач-эксперт', 'pro...","[{'id': '24', 'name': 'Врач'}]",2022-07-19T14:43:41+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/67921742,...,Полный день,Полная занятость,5951038.0,ГКУ МО Центр Внедрения Изменений Министерства ...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,67990296.0,Специалист по пожарной безопасности (п. Шатурт...,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'Внутренний контроль'}, {'name': 'Вв...","[{'id': '20.396', 'name': 'Эксплуатация', 'pro...","[{'id': '45', 'name': 'Инженер по охране труда...",2022-07-20T14:20:20+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/67990296,...,Полный день,Полная занятость,1434854.0,Федеральная мебельная компания Е1,NaN,NaN,NaN,NaN,NaN,NaN
4,4,68355727.0,Программист 1С,<strong>Обязанности:</strong> <ul> <li>Участие...,[{'name': '1С: Управление Производственным Пре...,"[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '96', 'name': 'Программист, разработчи...",2022-08-01T15:31:27+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68355727,...,Полный день,Полная занятость,1380.0,Новая столица,Егорьевск,Коломенское шоссе,2,55.360853,39.073364,"Егорьевск, Коломенское шоссе, 2"


In [8]:
new_df = df
new_df['description_2'] = new_df['description'].apply(lambda x: sim.clean_text(str(x)) )

In [9]:
new_df.head()

,Unnamed: 0,id,name,description,key_skills,specializations,professional_roles,published_at,apply_alternate_url,alternate_url,...,employment.name,employer.id,employer.name,address.city,address.street,address.building,address.lat,address.lng,address.raw,description_2
0,0,68153788.0,Преподаватель по классу гитары,<strong>Обязанности:</strong> <ul> <li>препода...,"[{'name': 'Грамотная речь'}, {'name': 'Способн...","[{'id': '14.60', 'name': 'Гуманитарные науки',...","[{'id': '132', 'name': 'Учитель, преподаватель...",2022-07-26T13:24:15+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68153788,...,Полная занятость,9047939.0,МУДО Егорьевская детская школа искусств,NaN,NaN,NaN,NaN,NaN,NaN,обязанность преподавание рамка дополнительный...
1,1,68357063.0,Продавец,<p><em>Вакансии открыты в разных микрорайонах ...,[],"[{'id': '17.302', 'name': 'FMCG, Товары народн...","[{'id': '97', 'name': 'Продавец-консультант, п...",2022-08-15T10:24:41+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68357063,...,Полная занятость,49357.0,"МАГНИТ, Розничная сеть",Шатура,Маршала Борзова,10,55.579923,39.521530,"Шатура, Маршала Борзова, 10",вакансия открытый разный микрорайон город во...
2,2,67921742.0,Врач-эндокринолог (г. Солнечногорск),<p><strong><em>Министерство Здравоохранения МО...,"[{'name': 'Пользователь ПК'}, {'name': 'Работа...","[{'id': '13.587', 'name': 'Врач-эксперт', 'pro...","[{'id': '24', 'name': 'Врач'}]",2022-07-19T14:43:41+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/67921742,...,Полная занятость,5951038.0,ГКУ МО Центр Внедрения Изменений Министерства ...,NaN,NaN,NaN,NaN,NaN,NaN,министерство здравоохранение мо приглашать ра...
3,3,67990296.0,Специалист по пожарной безопасности (п. Шатурт...,<strong>Обязанности:</strong> <ul> <li>Разрабо...,"[{'name': 'Внутренний контроль'}, {'name': 'Вв...","[{'id': '20.396', 'name': 'Эксплуатация', 'pro...","[{'id': '45', 'name': 'Инженер по охране труда...",2022-07-20T14:20:20+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/67990296,...,Полная занятость,1434854.0,Федеральная мебельная компания Е1,NaN,NaN,NaN,NaN,NaN,NaN,обязанность разработка внутренний нормативный ...
4,4,68355727.0,Программист 1С,<strong>Обязанности:</strong> <ul> <li>Участие...,[{'name': '1С: Управление Производственным Пре...,"[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '96', 'name': 'Программист, разработчи...",2022-08-01T15:31:27+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://hh.ru/vacancy/68355727,...,Полная занятость,1380.0,Новая столица,Егорьевск,Коломенское шоссе,2,55.360853,39.073364,"Егорьевск, Коломенское шоссе, 2",обязанность участие переход упп erp разра...


In [10]:
new_df.to_excel('Test2.xlsx')